In [2]:
import tensorflow as tf
import numpy as np
import cv2

# Load the pre-trained model
model_dir = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model'
model = tf.saved_model.load(model_dir)
vid_name= input("Enter video name :")
# Initialize video capture from the dashcam (or a video file)
# For dashcam, use 0 or the specific device index. For a file, replace '0' with 'path_to_video_file'
cap = cv2.VideoCapture(vid_name)  # Replace with 0 for real-time dashcam feed

# Check if the video capture is initialized properly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Define the confidence threshold
confidence_threshold = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Preprocess the frame
    input_tensor = tf.convert_to_tensor(frame_rgb)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run the model
    detections = model(input_tensor)

    # Process the output
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy().astype(np.int32)
    detection_scores = detections['detection_scores'][0].numpy()

    height, width, _ = frame.shape

    # Draw bounding boxes and labels on the frame
    for i in range(len(detection_boxes)):
        if detection_scores[i] >= confidence_threshold:
            box = detection_boxes[i]
            class_id = detection_classes[i]
            score = detection_scores[i]
            
            # Convert to pixel coordinates
            y_min, x_min, y_max, x_max = box
            x_min = int(x_min * width)
            x_max = int(x_max * width)
            y_min = int(y_min * height)
            y_max = int(y_max * height)
            
            # Draw the bounding box
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (125, 255, 25), 2)
            
            # Draw the label
            label = f'Class {class_id}: {score:.2f}'
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Real-time Object Detection', frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()
